<a href="https://colab.research.google.com/github/Gabones/tensor-flow/blob/main/00__TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neste notebook vamos abordar os aspectos mais fundamentais do TensorFlow

Mais especificamente:
* Introdução to tensors
* Obtendo informações de tensores
* Manipular tensores
* Tensores e Numpy
* Usar @tf.function (uma maneira de acelerar suas funções em Python)
* Usar GPUs com TensorFlow (ou TPUs)
* Exercícios

## Introdução a Tensores

In [ ]:
# Import TensorFlow
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.12.0


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the vector dimensions
vector.ndim

1

In [ ]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
matrix.ndim

2

In [ ]:
# Create another matrix with data type parameter (dtype)
another_matrix = tf.constant([[10.,7.],
                              [3.,2.],
                              [8.,9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
another_matrix.ndim

2

In [ ]:
# Let's create a tensor 3 matrix 2x3
tensor = tf.constant([
    [[1,2,3], [4,5,6]],
    [[7,8,9],[10,11,12]],
    [[13,14,15],[16,17,18]]
])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

O que criamos até agora:

* Escalar: um número simples
* Vetor: um array de números com direção (ex.: velocidade do vento e direção)
* Matriz: um array de duas dimensões
* Tensor: um array de n-dimensões

### Criando tensores com `tf.Variable`

In [ ]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])

print(changeable_tensor)
print(unchangeable_tensor)

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>
tf.Tensor([10  7], shape=(2,), dtype=int32)


In [ ]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7

TypeError: ignored

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
print(changeable_tensor)

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>


In [ ]:
# Let's try change our unchangeable tensor
# unchangeable_tensor[0] = 7 does not work
unchangeable_tensor[0].assign(7)

AttributeError: ignored

🔑 Note: Raramente você terá que decidir entre usar `tf.constant` ou `tf.Variable` para criar tensores uma vez que o TensorFlow faz isso pra você. Entretanto na dúvida use `tf.constant` e altere depois se for necessário.

### Criando tensores aleatórios

Random tensors are tensors of some arbitrary size which contain random numbers.

In [ ]:
# Create two random (but the same) tensors
rand_1 = tf.random.Generator.from_seed(7)
rand_1 = rand_1.normal(shape=[3,2])
print(rand_1)

rand_2 = tf.random.Generator.from_seed(7)
rand_2 = rand_2.normal(shape=[3,2])

print(rand_1)
print(rand_2)
print(rand_1 == rand_2)

tf.Tensor(
[[-1.3240396   0.28785667]
 [-0.8757901  -0.08857018]
 [ 0.69211644  0.84215707]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[-1.3240396   0.28785667]
 [-0.8757901  -0.08857018]
 [ 0.69211644  0.84215707]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[-1.3240396   0.28785667]
 [-0.8757901  -0.08857018]
 [ 0.69211644  0.84215707]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[ True  True]
 [ True  True]
 [ True  True]], shape=(3, 2), dtype=bool)


### Trocando a ordem dos elementos nos tensores

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])

print(not_shuffled.ndim)
print(not_shuffled)

2
tf.Tensor(
[[10  7]
 [ 3  4]
 [ 2  5]], shape=(3, 2), dtype=int32)


In [ ]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
shuffled = tf.random.shuffle(not_shuffled, seed=42)
print(shuffled)

tf.Tensor(
[[10  7]
 [ 3  4]
 [ 2  5]], shape=(3, 2), dtype=int32)


###⚒ **Exercício:** Ler a documentação do TensorFlow sobre random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed e praticar a geração de 5 tensores aleatórios e alterar a ordem deles.

Se quisermos ter todos os nossos tensores reordenados na mesma ordem, temos que usar simultaneamente uma semente aleatória de nível global e uma semente aleatória a nível de operação.

> Regra 4: If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [ ]:
tf.random.set_seed(7)
tens1 = tf.random.uniform(shape=[3,2], seed=7)
tens2 = tf.random.uniform(shape=[3,2], seed=7)
tens3 = tf.random.uniform(shape=[3,2], seed=7)
tens4 = tf.random.uniform(shape=[3,2], seed=7)
tens5 = tf.random.uniform(shape=[3,2], seed=7)

print(tens1)
print(tens2)
print(tens3)
print(tens4)
print(tens5)
print('-------------------------------')

tens1 = tf.random.shuffle(tens1, seed=42)
tens2 = tf.random.shuffle(tens2, seed=42)
tens3 = tf.random.shuffle(tens3, seed=42)
tens4 = tf.random.shuffle(tens4, seed=42)
tens5 = tf.random.shuffle(tens5, seed=42)

print(tens1)
print(tens2)
print(tens3)
print(tens4)
print(tens5)

tf.Tensor(
[[0.28619576 0.5422323 ]
 [0.503878   0.10653019]
 [0.89362943 0.6463729 ]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.6793382  0.34133482]
 [0.7055764  0.13367772]
 [0.16606605 0.32814598]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.32253122 0.16533732]
 [0.8505912  0.6394105 ]
 [0.12019205 0.9969933 ]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.3710593  0.99506664]
 [0.18928313 0.04732072]
 [0.09799647 0.12787962]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.41071463 0.39764488]
 [0.75160694 0.9127338 ]
 [0.06074905 0.5325992 ]], shape=(3, 2), dtype=float32)
-------------------------------
tf.Tensor(
[[0.28619576 0.5422323 ]
 [0.89362943 0.6463729 ]
 [0.503878   0.10653019]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.6793382  0.34133482]
 [0.7055764  0.13367772]
 [0.16606605 0.32814598]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.8505912  0.6394105 ]
 [0.32253122 0.16533732]
 [0.12019205 0.9969933 ]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.18928313 0.047320

###Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensor of all zeros
tf.zeros(shape=[3,4])

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Transformar NumPy arrays em tensores

A principal diferença entre NumPy arrays e tensores do TensorFlow é que os tensores podem rodar seus cálculos na GPU o que é muito mais rápido para computação númerica.

In [ ]:
# You can also turn Numpy arrays into tensors
# X = tf.constant(some_matrix) # letras maiúsculas para matrizes
# y = tf.constant(vector) # letras minúsculas para vetores
import numpy as np

In [ ]:
numpy_A = np.arange(1,25, dtype=np.int32)
print(numpy_A, '\n')

A = tf.constant(numpy_A)
print(A, '\n')

# a modificação do shape do tensor tem que ser um múltiplo do número de elementos
# do tensor original, numpy_A tem 24 elementos, 2 * 3 * 4 = 24
A = tf.constant(numpy_A, shape=[2,3,4]) # duas matrizes de 3x4
print(A, '\n')

A = tf.constant(numpy_A, shape=[3,8]) # uma matriz de 3x8
print(A, '\n')

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 

tf.Tensor([ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24], shape=(24,), dtype=int32) 

tf.Tensor(
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]], shape=(2, 3, 4), dtype=int32) 

tf.Tensor(
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]], shape=(3, 8), dtype=int32) 



###Getting information from tensors

* Shape `tensor.shape`
* Rank `tensor.ndim`
* Axis or dimension `tensor[0], tensor[:,1]`
* Size `tf.size(tensor)`

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4t = tf.zeros(shape=[2,3,4,5]) #2 conjuntos de 3 matrizes 4x5
print(rank_4t)

tf.Tensor(
[[[[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]]], shape=(2, 3, 4, 5), dtype=float32)


In [ ]:
rank_4t[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
print(rank_4t.shape, '\n')
print(rank_4t.ndim, '\n')
print(tf.size(rank_4t), '\n')

(2, 3, 4, 5) 

4 

tf.Tensor(120, shape=(), dtype=int32) 



In [ ]:
# Get varius attributes of our tensor
print('Datatype of every element: ', rank_4t.dtype)
print('Number of dimensions (rank): ', rank_4t.ndim)
print('Shape of tensor: ', rank_4t.shape)
print('Elements along the 0 axis: ', rank_4t.shape[0])
print('Elements along the last axis: ', rank_4t.shape[-1])
print('Total number of elements in our tensor: ', tf.size(rank_4t))
print('Total number of elements in our tensor: ', tf.size(rank_4t).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors
Tensors can be indexed just like Python lists.

In [ ]:
alguma_lista = [1, 2, 3, 4]
alguma_lista[:2]

[1, 2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4t[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
alguma_lista[:1]

[1]

In [ ]:
# Get the first element from each dimension except for the final one
print(rank_4t[:1, :1, :1])
print(rank_4t[:1, :1, :1, :])
print(rank_4t[:1, :1, :, :1])
print(rank_4t[:1, :, :1, :1])

tf.Tensor([[[[0. 0. 0. 0. 0.]]]], shape=(1, 1, 1, 5), dtype=float32)
tf.Tensor([[[[0. 0. 0. 0. 0.]]]], shape=(1, 1, 1, 5), dtype=float32)
tf.Tensor(
[[[[0.]
   [0.]
   [0.]
   [0.]]]], shape=(1, 1, 4, 1), dtype=float32)
tf.Tensor(
[[[[0.]]

  [[0.]]

  [[0.]]]], shape=(1, 3, 1, 1), dtype=float32)


In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2t = tf.constant([[10, 7],
                       [3, 4]])
rank_2t

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
rank_2t.shape, rank_2t.ndim

(TensorShape([2, 2]), 2)

In [ ]:
alguma_lista[-1]

4

In [ ]:
# Get the last item of each row of our rank 2 tensor
rank_2t[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [ ]:
# Add an extra dimension to our rank 2 tensor at the end
rank_3t = rank_2t[..., tf.newaxis]
print(rank_3t)

# Add an extra dimension to our rank 2 tensor in an arbitrary position
rank_3t = rank_2t[:, tf.newaxis, :]
print(rank_3t)

tf.Tensor(
[[[10]
  [ 7]]

 [[ 3]
  [ 4]]], shape=(2, 2, 1), dtype=int32)
tf.Tensor(
[[[10  7]]

 [[ 3  4]]], shape=(2, 1, 2), dtype=int32)


In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2t, axis=-1) # axis é o eixo onde a expansão será adicionada

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2t, axis=1)

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

### Aula 23 - Manipulating tensors (Tensor Operations)

Manipulating tensors with basic operations `+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addtion operator
tensor = tf.constant([[10,7], [3,4]])

tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Original tensor is unchanged because we didn't assign to tensor
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# If you want to change the original tensor
tensor = tensor + 10
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[200, 170],
       [130, 140]], dtype=int32)>

In [ ]:
# Subtraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# We can use the tensorflow built-in function too (faster in gpu)
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[200, 170],
       [130, 140]], dtype=int32)>

In [ ]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most commom tensor operation.
There are two rules our tensors (or matrices) need to fullfil if we're going to matrix multiply them:



1.   The inner dimensions must match
2.   The resulting matrix has the shape of the outer dimensions



In [ ]:
# Matrix multiplication in tensorflow

tf.matmul(tensor, tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]], dtype=int32)>

In [ ]:
# Matrix multiplication with Python operators

print(tensor)

print(tensor * tensor)

print(tensor @ tensor)

tf.Tensor(
[[20 17]
 [13 14]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[400 289]
 [169 196]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[621 578]
 [442 417]], shape=(2, 2), dtype=int32)


In [ ]:
tensor.shape

TensorShape([2, 2])

In [ ]:
# Create a tensor (3,2) tensor
X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])

# Create another (3,2) tensor
Y = tf.constant([[7,8],
                 [9,10],
                 [11,12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# Try to matrix multiply tensors of same shape
tf.matmul(X, Y)

InvalidArgumentError: ignored

In [ ]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [ ]:
# The inner dimensions match
X.shape, tf.reshape(Y, shape=(2,3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [ ]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [ ]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [ ]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Changing the X (3,2) tensor for a X (3,3) tensor
X = tf.constant([[5,0,3],
                 [3,7,9],
                 [3,5,2]])

# Changing the Y (3,2) tensor for a Y (3,2) tensor
Y = tf.constant([[4,7],
                 [6,8],
                 [8,1]])

X, Y

(<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[5, 0, 3],
        [3, 7, 9],
        [3, 5, 2]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[4, 7],
        [6, 8],
        [8, 1]], dtype=int32)>)

In [ ]:
tf.matmul(X, Y)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 44,  38],
       [126,  86],
       [ 58,  63]], dtype=int32)>

📖 Resource: Info and example of matrix multiplication: https://www.mathsisfun.com/algebra/matrix-multiplying.html

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [ ]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(X, tf.transpose(Y), axes=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Check the values of Y, reshape Y and transpose Y
print("Normal Y: ")
print(Y, "\n")

print("Y reshaped to (2,3): ")
print(tf.reshape(Y, (2, 3)), "\n")

print("Y transposed: ")
print(tf.transpose(Y))

Normal Y: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


Geralmente quando você estiver multiplicando dois tensores e as dimensões não forem compativeis você deve usar a transposta de um tensor para satisfazer as regras da mutiplicação, evite usar o reshape.

### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [ ]:
C = tf.constant([7,10])
C.dtype

tf.int32

In [ ]:
# Change to float32 to float16
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [ ]:
# Change to int32 to float32
E = tf.cast(C, dtype=tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E, E_float16.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float16)

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Get the absolute values
D = tf.constant([-7,-10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [ ]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the follow forms of aggregation:


*   Get the minimum
*   Get the maximum
*   Get the mean of a tensor
*   Get the sum of a tensor

In [ ]:
# Creating a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0,100,size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([79, 25, 70, 87, 73, 67, 34, 72, 46, 77, 48, 28, 63,  7, 60, 29, 27,
       33, 34,  3, 27, 56, 60, 47, 27, 40,  4, 57, 54, 22, 89, 80, 87, 41,
       81, 27, 44, 37, 62, 21, 32, 15, 66,  5, 15, 94, 25, 66, 31, 83])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the min of a tensor
print(tf.reduce_min(E))
print(np.min(E))

tf.Tensor(3, shape=(), dtype=int64)
3


In [ ]:
# Find the max of a tensor
print(tf.reduce_max(E))
print(np.max(E))

tf.Tensor(94, shape=(), dtype=int64)
94


In [ ]:
# Find the mean
print(tf.reduce_mean(E))
print(np.mean(E))

tf.Tensor(47, shape=(), dtype=int64)
47.14


In [ ]:
# Find the sum
print(tf.reduce_sum(E))
print(np.sum(E))

tf.Tensor(2357, shape=(), dtype=int64)
2357


In [ ]:
# Variance of the tensor
print(tf.math.reduce_variance(tf.cast(E, dtype=tf.float32)))
print(np.var(E))

tf.Tensor(632.0004, shape=(), dtype=float32)
632.0004000000001


In [ ]:
# Standard deviation numpy
print(tf.math.reduce_std(tf.cast(E, dtype=tf.float32)))
print(np.std(E))

tf.Tensor(25.139618, shape=(), dtype=float32)
25.139618135524653


### Find the positional maximum and minimum

In [ ]:
# Creating a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F)
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor to get started
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.7402308 , 0.33938193, 0.5692506 , 0.44811392, 0.29285502,
           0.4260056 , 0.62890387, 0.691061  , 0.30925727, 0.89236605,
           0.66396606, 0.30541587, 0.8724164 , 0.1025728 , 0.56819403,
           0.25427842, 0.7253866 , 0.4770788 , 0.46289814, 0.88944995,
           0.6792555 , 0.09752727, 0.01609659, 0.4876021 , 0.5832968 ,
           0.41212583, 0.731905  , 0.93418944, 0.5298122 , 0.9664817 ,
           0.88391197, 0.10578597, 0.44439578, 0.7851516 , 0.47332513,
           0.89893615, 0.04290593, 0.8717004 , 0.6068529 , 0.12963045,
           0.4527359 , 0.24573493, 0.34777248, 0.582147  , 0.82298195,
           0.82862926, 0.877372  , 0.5319803 , 0.03594303, 0.03986669]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.7402308 , 0.33938193, 0.5692506 , 0.44811392, 0.29285502,
        0.4260056 , 0.62890387, 0.691061  , 0.30925727, 0.89236605,
        0.66396606, 0.30541587, 0.8724164 , 0.1025728 , 0.56819403,
        0.25427842, 0.7253866 , 0.4770788 , 0.46289814, 0.88944995,
        0.6792555 , 0.09752727, 0.01609659, 0.4876021 , 0.5832968 ,
        0.41212583, 0.731905  , 0.93418944, 0.5298122 , 0.9664817 ,
        0.88391197, 0.10578597, 0.44439578, 0.7851516 , 0.47332513,
        0.89893615, 0.04290593, 0.8717004 , 0.6068529 , 0.12963045,
        0.4527359 , 0.24573493, 0.34777248, 0.582147  , 0.82298195,
        0.82862926, 0.877372  , 0.5319803 , 0.03594303, 0.03986669],
       dtype=float32)>,
 TensorShape([50]))

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0,1,2,3] # could be red, green, blue, purple

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="on", off_value="off")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'on', b'off', b'off', b'off'],
       [b'off', b'on', b'off', b'off'],
       [b'off', b'off', b'on', b'off'],
       [b'off', b'off', b'off', b'on']], dtype=object)>

### Squaring, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Find the squareroot
tf.sqrt(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# Find the log of a tensor
tf.math.log(tf.cast(H,dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensor and NumPy

TensorFlow interacts beatifully with NumPy array

In [ ]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
J.numpy()[1]

7.0

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

Um tensor do tensorflow é otimizado para rodar em uma GPU, por este motivo não usamos os arrays da biblioteca numpy no lugar deles.

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Fri Apr 14 03:19:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

🔑 Note: If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenver possible.